### Criando a classe dos dados:

In [ ]:
import random
import json

In [2]:
class Sentiment:
  NEGATIVE = "NEGATIVE"
  NEUTRAL = "NEUTRAL"
  POSITIVE = "POSITIVE"

In [48]:
class Review:
  def __init__(self, text, score):
    self.text = text
    self.score = score
    self.sentiment = self.get_sentiment()

  def get_sentiment(self):
    if self.score <= 2:
      return Sentiment.NEGATIVE
    elif self.score == 3:
      return Sentiment.NEUTRAL
    else: #score = 4 ou 5
      return Sentiment.POSITIVE
    

# queremos que a quantidade de negativos e positivos seja igual no conjunto de treino, 
# para isso, vamos analisar quais reviews são positivas e quais são negativas no nosso
# conjunto de treino
class ReviewContainer:
  def __init__(self, reviews):
    self.reviews = reviews

  def get_text(self):
    return [x.text for x in self.reviews]

  def get_sentiment(self):
    return [x.sentiment for x in self.reviews]

  def evenly_distribute(self):
    negative = list(filter(lambda x: x.sentiment == Sentiment.NEGATIVE, self.reviews))
    positive = list(filter(lambda x: x.sentiment == Sentiment.POSITIVE, self.reviews))
    
    positive_shrunk = positive[:len(negative)] # escolhendo a quantidade de reviews positivas como sendo igual a quantidade de 
    # reviews negativas
    
    self.reviews = negative + positive_shrunk # as reviews finais serão as reviews negativas + positivas
    random.seed(42)
    random.shuffle(self.reviews) # aleatoriezando as reviews

### Carregando os dados:

In [4]:
file_name = 'data/Books_small_10000.json'

reviews = []

with open(file_name) as f:
  for line in f:
    review = json.loads(line)
    reviews.append(Review(review['reviewText'], review['overall']))

print(reviews[5].text)
print(reviews[5].score)

I hoped for Mia to have some peace in this book, but her story is so real and raw.  Broken World was so touching and emotional because you go from Mia's trauma to her trying to cope.  I love the way the story displays how there is no "just bouncing back" from being sexually assaulted.  Mia showed us how those demons come for you every day and how sometimes they best you. I was so in the moment with Broken World and hurt with Mia because she was surrounded by people but so alone and I understood her feelings.  I found myself wishing I could give her some of my courage and strength or even just to be there for her.  Thank you Lizzy for putting a great character's voice on a strong subject and making it so that other peoples story may be heard through Mia's.
5.0


### Preparando os dados

In [ ]:
from sklearn.model_selection import train_test_split

training, test = train_test_split(reviews, test_size = 0.33, random_state = 42)

train_container = ReviewContainer(training)

test_container = ReviewContainer(test)

In [58]:
train_container.evenly_distribute() # igualando a quantidade de reviews positivas e negativas:

train_x = train_container.get_text()
train_y = train_container.get_sentiment()

test_x = test_container.get_text()
test_y = test_container.get_sentiment()

print('Quantidade de reviews POSITIVAS: ', train_y.count(Sentiment.POSITIVE))
print('Quantidade de reviews NEGATIVAS: ',train_y.count(Sentiment.NEGATIVE))

Quantidade de reviews POSITIVAS:  436
Quantidade de reviews NEGATIVAS:  436


##### Vetorização utilizando Bags of Words

![alt text](Screenshot_1.jpg "Title")

In [59]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer()

# transformando os textos em train_x em vetores numéricos
#
train_x_vectors = vectorizer.fit_transform(train_x)
# vectorizer.fit aprende o dicionário com todas as palavras 
# vectorizer.transform transforma a string num vetor utilizando o dicionário aprendido
# vectorizer.fit_transform faz ambas as coisas

test_x_vectors = vectorizer.transform(test_x)

print(train_x[0])
print(train_x_vectors[0])

Excellent children's story. Read it to children, now to grandchildren.
<Compressed Sparse Row sparse matrix of dtype 'int64'
	with 8 stored elements and shape (1, 8906)>
  Coords	Values
  (0, 2814)	1
  (0, 1409)	2
  (0, 7533)	1
  (0, 6392)	1
  (0, 4277)	1
  (0, 8052)	2
  (0, 5426)	1
  (0, 3477)	1


#### Classificação

In [60]:
from sklearn.linear_model import LogisticRegression

# criando um classificador
clf_log = LogisticRegression()

# ajustando os vetores
clf_log.fit(train_x_vectors, train_y)

# queremos prever, a partir do nosso modelo, se o seguinte texto, que é positivo,
#  é positivo ou negativo:
print(train_x[0])
print(train_y[0])

# resultado previsto:
print("O texto é: ", clf_log.predict(train_x_vectors[0]))

Excellent children's story. Read it to children, now to grandchildren.
POSITIVE
O texto é:  ['POSITIVE']


#### Avaliação do modelo

In [61]:
# acurácia média do modelo, ou seja, queremos saber o quão bem, em média,
#  os vetores de treino, em test_x_vectors, predizem os vetores test_y
clf_log.score(test_x_vectors, test_y)

0.7451515151515151

In [62]:
# Calculando o F1 score, que é uma medida que leva em conta a quantidade de falsos
# positivos, falsos negativos e positivos verdadeiros, é, de certa forma uma medida mais 
# precisa do que a medida score

from sklearn.metrics import f1_score

# recebe o valor real e o valor predito, retorna o quanto o modelo está acertando para
# cada classe que queremos prever
#
f1_score(test_y, clf_log.predict(test_x_vectors), average=None, 
         labels=[Sentiment.POSITIVE, Sentiment.NEUTRAL, Sentiment.NEGATIVE])

array([0.87834736, 0.        , 0.31136581])

Notamos que o nosso modelo tem uma boa capacidade para prever valores positivos, porém peca em prever valores negativos e neutros.

Notamos que a quantidade de valores POSITIVOS no nosso conjunto de treinamento é muito maior que a quantidade de valores NEGATIVOS, por esse motivo, a capacidade do modelo de prever valores negativos é fortemente prejudicada. Para resolver isso, aumentaremos o banco de dados.

In [12]:
print('Quantidade de valores POSITIVOS no conjunto de treino: ',
      train_y.count(Sentiment.POSITIVE))

print('Quantidade de valores NEGATIVOS no conjunto de treino: ',
      train_y.count(Sentiment.NEGATIVE))

Quantidade de valores POSITIVOS no conjunto de treino:  5611
Quantidade de valores NEGATIVOS no conjunto de treino:  436


O objetivo é distribuir igualmente a quantidade de reviews positivas e negativas para o nosso conjunto de teste e, dessa forma, não viesar o nosso modelo, buscando uma melhor qualidade de previsão para as opiniões positivas e negativas